In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def data():
    training = pd.read_csv("./training_preprocessed")
    validation = pd.read_csv("./validation_preprocessed")
    test = pd.read_csv("./test_preprocessed")
    X_train = training.drop(columns="y")
    y_train = training["y"]
    X_valid = validation.drop(columns="y")
    y_valid = validation["y"]
    X_test = test
    return X_train, y_train, X_valid, y_valid, X_test

In [4]:
X_train, y_train, X_valid, y_valid, X_test = data()

In [19]:
X_valid

,0,1,2,3,4,5,6,7,8,9,...,299,300,301,302,303,304,305,306,307,308
0,0.543460,0.236888,1.393064,-0.526788,0.683623,0.340810,-0.574965,0.811542,-0.292288,-1.003178,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.215827,2.033324,-1.508583,-0.526788,-0.275562,-1.060661,-0.574965,0.452125,-0.292288,-0.296999,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.159251,-0.410041,-0.057760,0.372756,0.154418,-0.432415,0.830062,0.447517,-0.292288,-0.344530,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1.359905,0.234562,-0.057760,-0.526788,0.551322,0.195830,0.901691,0.908308,-0.292288,-0.548236,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.417063,-0.458178,-0.783171,-0.526788,1.113603,1.017382,-0.574965,1.251597,-0.292288,-1.023549,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1.503983,2.646118,-0.783171,-0.526788,-0.540165,-1.447274,-0.574965,1.426697,-0.292288,-1.272522,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
288,0.543460,0.149527,-0.783171,0.372756,-0.507089,-1.398947,-0.574965,0.154915,-0.292288,-0.036708,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
289,-0.417063,0.378626,0.667652,1.272300,-1.896254,-0.722375,-0.574965,-0.107735,-0.292288,-0.163459,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
290,-0.032854,1.358238,-0.057760,-0.526788,-0.143260,-0.432415,1.403093,0.210210,-0.292288,1.334909,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
wandb.init(project="house-price-prediction", entity="luiscostigan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luiscostigan (use `wandb login --relogin` to force relogin)


In [6]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
    
    return model

In [7]:
from sklearn.model_selection import KFold

# Instantiate the validator, have it spit out the indices, generate batches from the indices, and feed those batches to the model.
kf = KFold(n_splits=5)
kf.get_n_splits(X_train, y_train)

5

Currently trying to implement RepeatedKFold cross validation with the Keras Sequential model, while recording model results on WandB.

Sometimes I am getting a failed-1 error from WandB.
    - 

In [8]:
from wandb.keras import WandbCallback

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "random",
  "parameters": {
    "dropout1": {
      "min": 0.0,
      "max": 0.8
    },
    "epochs": {
      "values": [30,50,100]
    },
    "batch_size": {
      "values": [16, 64, 128, 512]
    }
  }
}

config_defaults = {
  "dropout1": 0.5,
  "epochs": 50,
  "batch_size": 64
}

def train():

  mae_per_fold = []
  loss_per_fold = []
  fold_no = 1

  # Go through each split, and get the index number for each (currently 50, 10 x 5 repeated k-fold) 
  for train, test in kf.split(X_train, y_train):

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      # Recreate the model each time for each new batch
      model = None # Not sure if this step is necessary
      model = create_model()

      # Fit model on new batches
      model.fit(
        np.asarray(X_train), 
        y_train, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(X_train),y_train)
        )
      
      # Generate data for each
      scores = model.evaluate(np.asarray(X_train), y_train, callbacks=[WandbCallback()])
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
      mae_per_fold.append(scores[1])
      loss_per_fold.append(scores[0])

      # Increase fold number
      fold_no = fold_no + 1

      # == Provide average scores ==
      print('------------------------------------------------------------------------')
      print('Score per fold')
      for i in range(0, len(mae_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {mae_per_fold[i]}%')
      print('------------------------------------------------------------------------')
      print('Average scores for all folds:')
      print(f'> MAE: {np.mean(mae_per_fold)} (+- {np.std(mae_per_fold)})')
      print(f'> Loss: {np.mean(loss_per_fold)}')
      print('------------------------------------------------------------------------')

      wandb.log({
        "Mean Validation MAE (all folds)": np.mean(mae_per_fold),
        "Mean Validation Loss (all folds)": np.mean(loss_per_fold) 
        })

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 10
wandb.agent(keras_sequential_sweep_1, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: cp52f928
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/cp52f928


wandb: Agent Starting Run: vhc2rlyx with config:
wandb: 	batch_size: 16
wandb: 	dropout1: 0.20627260536931216
wandb: 	epochs: 30


37/37 [==============================] - 0s 3ms/step - loss: 463129216.0000 - mae: 12170.9697
Score for fold 1: loss of 463129216.0; mae of 12170.9697265625
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 463129216.0 - Accuracy: 12170.9697265625%
------------------------------------------------------------------------
Average scores for all folds:
> MAE: 12170.9697265625 (+- 0.0)
> Loss: 463129216.0
------------------------------------------------------------------------


Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),463129216.0
Mean Validation MAE (all folds),12170.96973
best_epoch,29
best_val_loss,463129184.0


...

Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,██▇▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▇▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),479496992.0
Mean Validation MAE (all folds),12868.09375
best_epoch,49
best_val_loss,471004480.0


37/37 [==============================] - 0s 2ms/step - loss: 487222496.0000 - mae: 13082.9092
Score for fold 4: loss of 487222496.0; mae of 13082.9091796875
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 489506912.0 - Accuracy: 13148.0615234375%
------------------------------------------------------------------------
> Fold 2 - Loss: 477979520.0 - Accuracy: 12858.9521484375%
------------------------------------------------------------------------
> Fold 3 - Loss: 471004544.0 - Accuracy: 12597.267578125%
------------------------------------------------------------------------
> Fold 4 - Loss: 487222496.0 - Accuracy: 13082.9091796875%
------------------------------------------------------------------------
Average scores for all folds:
> MAE: 12921.797607421875 (+- 215.88283282779506)
> Loss: 481428368.0
-------------------------------------------------------

Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,██▇▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▇▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),481428368.0
Mean Validation MAE (all folds),12921.79761
best_epoch,46
best_val_loss,485625856.0


37/37 [==============================] - 0s 2ms/step - loss: 476419104.0000 - mae: 12716.4170
Score for fold 5: loss of 476419104.0; mae of 12716.4169921875
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 489506912.0 - Accuracy: 13148.0615234375%
------------------------------------------------------------------------
> Fold 2 - Loss: 477979520.0 - Accuracy: 12858.9521484375%
------------------------------------------------------------------------
> Fold 3 - Loss: 471004544.0 - Accuracy: 12597.267578125%
------------------------------------------------------------------------
> Fold 4 - Loss: 487222496.0 - Accuracy: 13082.9091796875%
------------------------------------------------------------------------
> Fold 5 - Loss: 476419104.0 - Accuracy: 12716.4169921875%
------------------------------------------------------------------------
Average scores for all 

Mean Validation Loss (all folds),▁
Mean Validation MAE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▆▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,██▇▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▇▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),480426515.2
Mean Validation MAE (all folds),12880.72148
best_epoch,49
best_val_loss,476419072.0


Train the model with Hyperopt to find best parameters.
Retrain model with new best parameters.
Make predictions.

In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [9]:
# Enter best params from sweep
best_params = {
    "dropout": 0.4863,
    "epochs": 100,
    "batch_size": 16
}

def make_predictions(best_params):
    
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss="mse", metrics=["mae"])
              
    model.fit(X_train, y_train, epochs=best_params.get("epochs"),
              batch_size=best_params.get("batch_size"), verbose = 0)
    
    preds  = model.predict(X_test, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [10]:
predictions = make_predictions(best_params)

In [14]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,299,300,301,302,303,304,305,306,307,308
0,0.543460,0.521305,-0.783171,0.372756,-0.341712,-1.157314,-0.574965,0.071973,0.595904,-0.661405,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.591487,0.920145,-0.057760,0.372756,-0.440939,-1.302294,0.020106,1.120271,-0.292288,-0.353584,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.255303,0.859635,-0.783171,-0.526788,0.849000,0.630770,-0.574965,0.816150,-0.292288,-0.962437,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.447408,0.224645,-0.057760,0.372756,0.882075,0.630770,-0.464767,0.380703,-0.292288,-0.539182,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1.233507,-1.117361,1.393064,-0.526788,0.683623,0.340810,-0.574965,-0.400337,-0.292288,1.029350,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,-2.290083,-2.964834,-1.508583,1.272300,-0.044034,-0.722375,-0.574965,-1.006277,-0.292288,-0.036708,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1455,-2.290083,-3.007496,-1.508583,-0.526788,-0.044034,-0.722375,-0.574965,-0.425681,-0.292288,-0.607084,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1456,4.385553,1.577164,-0.783171,1.272300,-0.374788,0.534116,-0.574965,1.813761,-0.292288,-1.272522,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1457,-0.321010,0.312870,-0.783171,-0.526788,0.683623,0.340810,-0.574965,-0.229845,-0.292288,0.028930,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Create Submissions

In [20]:
d = {"Id":X_test.index,"SalePrice":predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

In [21]:
submission["Id"] = submission["Id"] + 1461

In [23]:
submission.to_csv("submission.csv",index=False)

## Blend Predictions

First, try taking the mean of predictions from the decision tree algorithm and neural net algorithm.

In [32]:
# Read CSVs
decision_tree_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/sklearn/submission.csv")
neural_net_predictions = pd.read_csv("/root/data-science-projects-1/house-price-prediction/Models/tensorflow/submission.csv")

# Merge CSVs on Id column
decision_tree_predictions["neural_net_predictions"] = neural_net_predictions["SalePrice"]

# Rename decision tree predictions column
decision_tree_predictions = decision_tree_predictions.rename(columns={"SalePrice":"decision_tree_predictions"})

# Add new column with mean
decision_tree_predictions["SalePrice"] = decision_tree_predictions[["neural_net_predictions","decision_tree_predictions"]].mean(axis=1)

# Drop other columns
decision_tree_predictions = decision_tree_predictions[["Id","SalePrice"]]

mean_predictions = decision_tree_predictions

In [33]:
mean_predictions

,Id,SalePrice
0,1461,127361.134067
1,1462,158052.584090
2,1463,181674.855717
3,1464,188875.924001
4,1465,200818.447191
...,...,...
1454,2915,82795.660323
1455,2916,78708.973134
1456,2917,161475.439244
1457,2918,117693.042804


In [34]:
mean_predictions.to_csv("submission_mean.csv",index=False)

Next, try defining a meta-model to best blend predictions.

In [ ]:
# You currently have the following problems:
# 1) You have trained and validated using k-fold validation on a training set that was already split into a validation set.
# 2) You need to keep a "hold-out" data set to validate merged predictions, that was not used at all in the DT and NN models.